In [ ]:
import math
import random

import numpy
import scipy
import scipy.stats

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

%matplotlib inline

In [ ]:
def recommendNumberOfRepetitions(values, targetRatio = 0.2):
    standardErrors = [ math.sqrt(numpy.var(runs)/len(runs)) for runs in values]
    measuredMeans = [numpy.mean(runs) for runs in values]
    diffsToNext = [abs(measuredMeans[i] - measuredMeans[i+1]) for i in range(len(measuredMeans)-1)]
    repetitionsForRun = [len(runs) for runs in values]

    factorLambda = lambda pair: (pair[0]/(pair[1]*targetRatio))**2
    factors = list(map(factorLambda, zip(standardErrors,diffsToNext)))
    correctionFactor = scipy.stats.mstats.gmean(factors)
    
    return math.ceil(numpy.mean(repetitionsForRun)*correctionFactor)

In [ ]:
def generateExampleMeasurements(nList, alpha, beta, repetitions, variance):
    logTrueMeans = [alpha*n**beta for n in nList]
    times = [[random.lognormvariate(math.log(mean), variance) for i in range(repetitions)] for mean in logTrueMeans]
    return times

In [ ]:
def sampleAdditional(times, fidelity, blackBoxFunction):
    recommendedRepetitions = recommendNumberOfRepetitions(times,fidelity)
    repetitions = len(times[0])
    print("Ratio  ", fidelity, ": ", end='')
    while recommendedRepetitions > repetitions:
        # More repetitions recommended to smooth out plot
        print("Taking ", recommendedRepetitions-repetitions, " additional samples per point.")
        additional = blackBoxFunction(recommendedRepetitions-repetitions)
        times = list(map(lambda pair: pair[0]+pair[1], zip(times,additional)))
        #times = [pair[0]+pair[1] for pair in zip(times,additional)]
        repetitions = recommendedRepetitions
        recommendedRepetitions = recommendNumberOfRepetitions(times,fidelity)
    smoothedMeans = [numpy.mean(runs) for runs in times]
    plt.plot(nList, smoothedMeans)
    print(repetitions, " repetitions were sufficient.")
    return times

In [ ]:
nList = [2**(10 + e/5) for e in range(10,30)]

alpha = 0.001
beta = 1.1

repetitions = 3
variance = 0.5
times = generateExampleMeasurements(nList, alpha, beta, repetitions, variance)

def blackBoxFunction(repetitions):
    return generateExampleMeasurements(nList, alpha, beta, repetitions, variance)

In [ ]:
measuredMeans = [numpy.mean(runs) for runs in times]

plt.plot(nList, measuredMeans)

In [ ]:
times = sampleAdditional(times, 1, blackBoxFunction)

In [ ]:
times = sampleAdditional(times, 0.5, blackBoxFunction)

In [ ]:
times = sampleAdditional(times, 0.3, blackBoxFunction)

In [ ]:
times = sampleAdditional(times, 0.2, blackBoxFunction)

In [ ]:
times = sampleAdditional(times, 0.1, blackBoxFunction)